In [1]:
import pandas as pd

import nltk 
import string
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel


In [78]:
xl = pd.ExcelFile('data_for_content_based - Ext.xlsx')
df = xl.parse('Sheet1')
df = df.drop(columns=['Year', 'Ratings', 'No. of Employees', 'Industry'])


In [79]:
def remove_punct(text):   
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
    
df['Txt_punct'] = df['Overview'].apply(lambda x: remove_punct(x))
df.head(10)
    

,id,Name,Location,Overview,Txt_punct
0,1,Acclinate Genetics,"Huntsville, AL","We provide biopharmaceutical companies, contra...",We provide biopharmaceutical companies contrac...
1,2,Activ Surgical,"Boston, MA",Activ Surgical is focused on providing global ...,Activ Surgical is focused on providing global ...
2,3,AIM Medical Robotics,"Greater Miami, FL",We are developing a robotic system for use in ...,We are developing a robotic system for use in ...
3,4,AMChart,"Austin, TX",The AmChart platform was created to bring thes...,The AmChart platform was created to bring thes...
4,5,AngioInsight Inc,"Ann Arbor, MN",AngioInsight is a seed-stage start-up bringing...,AngioInsight is a seedstage startup bringing t...
5,6,AniCell Biotech,"Chandler, AR",It uses provisionally-patented collection and ...,It uses provisionallypatented collection and p...
6,7,Appairy Medical,British Columbia,Appairy has advanced cardiac monitoring system...,Appairy has advanced cardiac monitoring system...
7,8,ARIZ Precision Medicine,"Davis, CA",ARIZ Precision Medicine (ARIZ) is developing p...,ARIZ Precision Medicine ARIZ is developing pot...
8,9,Articulate Labs,"Dallas, TX","Articulate Labs has developed KneeStim(TM), a ...",Articulate Labs has developed KneeStimTM a med...
9,10,Avitus Orthopaedics,"Shelton, CT","Avitus Orthopaedics, Inc. is a medical device ...",Avitus Orthopaedics Inc is a medical device st...


In [80]:
# Tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text
    
df['Txt_tokenized'] = df['Txt_punct'].apply(lambda x: tokenization(x.lower()))
df.head(10)  
    

,id,Name,Location,Overview,Txt_punct,Txt_tokenized
0,1,Acclinate Genetics,"Huntsville, AL","We provide biopharmaceutical companies, contra...",We provide biopharmaceutical companies contrac...,"[we, provide, biopharmaceutical, companies, co..."
1,2,Activ Surgical,"Boston, MA",Activ Surgical is focused on providing global ...,Activ Surgical is focused on providing global ...,"[activ, surgical, is, focused, on, providing, ..."
2,3,AIM Medical Robotics,"Greater Miami, FL",We are developing a robotic system for use in ...,We are developing a robotic system for use in ...,"[we, are, developing, a, robotic, system, for,..."
3,4,AMChart,"Austin, TX",The AmChart platform was created to bring thes...,The AmChart platform was created to bring thes...,"[the, amchart, platform, was, created, to, bri..."
4,5,AngioInsight Inc,"Ann Arbor, MN",AngioInsight is a seed-stage start-up bringing...,AngioInsight is a seedstage startup bringing t...,"[angioinsight, is, a, seedstage, startup, brin..."
5,6,AniCell Biotech,"Chandler, AR",It uses provisionally-patented collection and ...,It uses provisionallypatented collection and p...,"[it, uses, provisionallypatented, collection, ..."
6,7,Appairy Medical,British Columbia,Appairy has advanced cardiac monitoring system...,Appairy has advanced cardiac monitoring system...,"[appairy, has, advanced, cardiac, monitoring, ..."
7,8,ARIZ Precision Medicine,"Davis, CA",ARIZ Precision Medicine (ARIZ) is developing p...,ARIZ Precision Medicine ARIZ is developing pot...,"[ariz, precision, medicine, ariz, is, developi..."
8,9,Articulate Labs,"Dallas, TX","Articulate Labs has developed KneeStim(TM), a ...",Articulate Labs has developed KneeStimTM a med...,"[articulate, labs, has, developed, kneestimtm,..."
9,10,Avitus Orthopaedics,"Shelton, CT","Avitus Orthopaedics, Inc. is a medical device ...",Avitus Orthopaedics Inc is a medical device st...,"[avitus, orthopaedics, inc, is, a, medical, de..."


In [81]:
# Remove stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
    
def remove_stopwords(text):
    text = [word for word in text if word not in stop_words]
    return text
        
df['Txt_nonstop'] = df['Txt_tokenized'].apply(lambda x: remove_stopwords(x))
df.head(10)    

,id,Name,Location,Overview,Txt_punct,Txt_tokenized,Txt_nonstop
0,1,Acclinate Genetics,"Huntsville, AL","We provide biopharmaceutical companies, contra...",We provide biopharmaceutical companies contrac...,"[we, provide, biopharmaceutical, companies, co...","[provide, biopharmaceutical, companies, contra..."
1,2,Activ Surgical,"Boston, MA",Activ Surgical is focused on providing global ...,Activ Surgical is focused on providing global ...,"[activ, surgical, is, focused, on, providing, ...","[activ, surgical, focused, providing, global, ..."
2,3,AIM Medical Robotics,"Greater Miami, FL",We are developing a robotic system for use in ...,We are developing a robotic system for use in ...,"[we, are, developing, a, robotic, system, for,...","[developing, robotic, system, use, neurosurgic..."
3,4,AMChart,"Austin, TX",The AmChart platform was created to bring thes...,The AmChart platform was created to bring thes...,"[the, amchart, platform, was, created, to, bri...","[amchart, platform, created, bring, datasets, ..."
4,5,AngioInsight Inc,"Ann Arbor, MN",AngioInsight is a seed-stage start-up bringing...,AngioInsight is a seedstage startup bringing t...,"[angioinsight, is, a, seedstage, startup, brin...","[angioinsight, seedstage, startup, bringing, b..."
5,6,AniCell Biotech,"Chandler, AR",It uses provisionally-patented collection and ...,It uses provisionallypatented collection and p...,"[it, uses, provisionallypatented, collection, ...","[uses, provisionallypatented, collection, proc..."
6,7,Appairy Medical,British Columbia,Appairy has advanced cardiac monitoring system...,Appairy has advanced cardiac monitoring system...,"[appairy, has, advanced, cardiac, monitoring, ...","[appairy, advanced, cardiac, monitoring, syste..."
7,8,ARIZ Precision Medicine,"Davis, CA",ARIZ Precision Medicine (ARIZ) is developing p...,ARIZ Precision Medicine ARIZ is developing pot...,"[ariz, precision, medicine, ariz, is, developi...","[ariz, precision, medicine, ariz, developing, ..."
8,9,Articulate Labs,"Dallas, TX","Articulate Labs has developed KneeStim(TM), a ...",Articulate Labs has developed KneeStimTM a med...,"[articulate, labs, has, developed, kneestimtm,...","[articulate, labs, developed, kneestimtm, medi..."
9,10,Avitus Orthopaedics,"Shelton, CT","Avitus Orthopaedics, Inc. is a medical device ...",Avitus Orthopaedics Inc is a medical device st...,"[avitus, orthopaedics, inc, is, a, medical, de...","[avitus, orthopaedics, inc, medical, device, s..."


In [82]:
ps = nltk.PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([i for i in text])
    return stem_text
    
df['cleaned_overview'] = df['Txt_nonstop'].apply(lambda x: word_stemmer(x))
df.head(10)    

,id,Name,Location,Overview,Txt_punct,Txt_tokenized,Txt_nonstop,cleaned_overview
0,1,Acclinate Genetics,"Huntsville, AL","We provide biopharmaceutical companies, contra...",We provide biopharmaceutical companies contrac...,"[we, provide, biopharmaceutical, companies, co...","[provide, biopharmaceutical, companies, contra...",provide biopharmaceutical companies contract r...
1,2,Activ Surgical,"Boston, MA",Activ Surgical is focused on providing global ...,Activ Surgical is focused on providing global ...,"[activ, surgical, is, focused, on, providing, ...","[activ, surgical, focused, providing, global, ...",activ surgical focused providing global access...
2,3,AIM Medical Robotics,"Greater Miami, FL",We are developing a robotic system for use in ...,We are developing a robotic system for use in ...,"[we, are, developing, a, robotic, system, for,...","[developing, robotic, system, use, neurosurgic...",developing robotic system use neurosurgical pr...
3,4,AMChart,"Austin, TX",The AmChart platform was created to bring thes...,The AmChart platform was created to bring thes...,"[the, amchart, platform, was, created, to, bri...","[amchart, platform, created, bring, datasets, ...",amchart platform created bring datasets market...
4,5,AngioInsight Inc,"Ann Arbor, MN",AngioInsight is a seed-stage start-up bringing...,AngioInsight is a seedstage startup bringing t...,"[angioinsight, is, a, seedstage, startup, brin...","[angioinsight, seedstage, startup, bringing, b...",angioinsight seedstage startup bringing benefi...
5,6,AniCell Biotech,"Chandler, AR",It uses provisionally-patented collection and ...,It uses provisionallypatented collection and p...,"[it, uses, provisionallypatented, collection, ...","[uses, provisionallypatented, collection, proc...",uses provisionallypatented collection processi...
6,7,Appairy Medical,British Columbia,Appairy has advanced cardiac monitoring system...,Appairy has advanced cardiac monitoring system...,"[appairy, has, advanced, cardiac, monitoring, ...","[appairy, advanced, cardiac, monitoring, syste...",appairy advanced cardiac monitoring systems co...
7,8,ARIZ Precision Medicine,"Davis, CA",ARIZ Precision Medicine (ARIZ) is developing p...,ARIZ Precision Medicine ARIZ is developing pot...,"[ariz, precision, medicine, ariz, is, developi...","[ariz, precision, medicine, ariz, developing, ...",ariz precision medicine ariz developing potent...
8,9,Articulate Labs,"Dallas, TX","Articulate Labs has developed KneeStim(TM), a ...",Articulate Labs has developed KneeStimTM a med...,"[articulate, labs, has, developed, kneestimtm,...","[articulate, labs, developed, kneestimtm, medi...",articulate labs developed kneestimtm medical d...
9,10,Avitus Orthopaedics,"Shelton, CT","Avitus Orthopaedics, Inc. is a medical device ...",Avitus Orthopaedics Inc is a medical device st...,"[avitus, orthopaedics, inc, is, a, medical, de...","[avitus, orthopaedics, inc, medical, device, s...",avitus orthopaedics inc medical device startup...


In [83]:
cleaned_df = df
cleaned_df['Overview'] = df['cleaned_overview']
cleaned_df.head(1)['Overview']
  

0    provide biopharmaceutical companies contract r...
Name: Overview, dtype: object

In [85]:
cleaned_df

,id,Name,Location,Overview,Txt_punct,Txt_tokenized,Txt_nonstop,cleaned_overview
0,1,Acclinate Genetics,"Huntsville, AL",provide biopharmaceutical companies contract r...,We provide biopharmaceutical companies contrac...,"[we, provide, biopharmaceutical, companies, co...","[provide, biopharmaceutical, companies, contra...",provide biopharmaceutical companies contract r...
1,2,Activ Surgical,"Boston, MA",activ surgical focused providing global access...,Activ Surgical is focused on providing global ...,"[activ, surgical, is, focused, on, providing, ...","[activ, surgical, focused, providing, global, ...",activ surgical focused providing global access...
2,3,AIM Medical Robotics,"Greater Miami, FL",developing robotic system use neurosurgical pr...,We are developing a robotic system for use in ...,"[we, are, developing, a, robotic, system, for,...","[developing, robotic, system, use, neurosurgic...",developing robotic system use neurosurgical pr...
3,4,AMChart,"Austin, TX",amchart platform created bring datasets market...,The AmChart platform was created to bring thes...,"[the, amchart, platform, was, created, to, bri...","[amchart, platform, created, bring, datasets, ...",amchart platform created bring datasets market...
4,5,AngioInsight Inc,"Ann Arbor, MN",angioinsight seedstage startup bringing benefi...,AngioInsight is a seedstage startup bringing t...,"[angioinsight, is, a, seedstage, startup, brin...","[angioinsight, seedstage, startup, bringing, b...",angioinsight seedstage startup bringing benefi...
5,6,AniCell Biotech,"Chandler, AR",uses provisionallypatented collection processi...,It uses provisionallypatented collection and p...,"[it, uses, provisionallypatented, collection, ...","[uses, provisionallypatented, collection, proc...",uses provisionallypatented collection processi...
6,7,Appairy Medical,British Columbia,appairy advanced cardiac monitoring systems co...,Appairy has advanced cardiac monitoring system...,"[appairy, has, advanced, cardiac, monitoring, ...","[appairy, advanced, cardiac, monitoring, syste...",appairy advanced cardiac monitoring systems co...
7,8,ARIZ Precision Medicine,"Davis, CA",ariz precision medicine ariz developing potent...,ARIZ Precision Medicine ARIZ is developing pot...,"[ariz, precision, medicine, ariz, is, developi...","[ariz, precision, medicine, ariz, developing, ...",ariz precision medicine ariz developing potent...
8,9,Articulate Labs,"Dallas, TX",articulate labs developed kneestimtm medical d...,Articulate Labs has developed KneeStimTM a med...,"[articulate, labs, has, developed, kneestimtm,...","[articulate, labs, developed, kneestimtm, medi...",articulate labs developed kneestimtm medical d...
9,10,Avitus Orthopaedics,"Shelton, CT",avitus orthopaedics inc medical device startup...,Avitus Orthopaedics Inc is a medical device st...,"[avitus, orthopaedics, inc, is, a, medical, de...","[avitus, orthopaedics, inc, medical, device, s...",avitus orthopaedics inc medical device startup...


In [86]:
cleaned_df.to_csv('cleaneddf.csv')

In [90]:
df1 = pd.read_csv('cleaneddf.csv', index_col=0)
df1

,id,Name,Location,Overview,Txt_punct,Txt_tokenized,Txt_nonstop,cleaned_overview
0,1,Acclinate Genetics,"Huntsville, AL",provide biopharmaceutical companies contract r...,We provide biopharmaceutical companies contrac...,"['we', 'provide', 'biopharmaceutical', 'compan...","['provide', 'biopharmaceutical', 'companies', ...",provide biopharmaceutical companies contract r...
1,2,Activ Surgical,"Boston, MA",activ surgical focused providing global access...,Activ Surgical is focused on providing global ...,"['activ', 'surgical', 'is', 'focused', 'on', '...","['activ', 'surgical', 'focused', 'providing', ...",activ surgical focused providing global access...
2,3,AIM Medical Robotics,"Greater Miami, FL",developing robotic system use neurosurgical pr...,We are developing a robotic system for use in ...,"['we', 'are', 'developing', 'a', 'robotic', 's...","['developing', 'robotic', 'system', 'use', 'ne...",developing robotic system use neurosurgical pr...
3,4,AMChart,"Austin, TX",amchart platform created bring datasets market...,The AmChart platform was created to bring thes...,"['the', 'amchart', 'platform', 'was', 'created...","['amchart', 'platform', 'created', 'bring', 'd...",amchart platform created bring datasets market...
4,5,AngioInsight Inc,"Ann Arbor, MN",angioinsight seedstage startup bringing benefi...,AngioInsight is a seedstage startup bringing t...,"['angioinsight', 'is', 'a', 'seedstage', 'star...","['angioinsight', 'seedstage', 'startup', 'brin...",angioinsight seedstage startup bringing benefi...
5,6,AniCell Biotech,"Chandler, AR",uses provisionallypatented collection processi...,It uses provisionallypatented collection and p...,"['it', 'uses', 'provisionallypatented', 'colle...","['uses', 'provisionallypatented', 'collection'...",uses provisionallypatented collection processi...
6,7,Appairy Medical,British Columbia,appairy advanced cardiac monitoring systems co...,Appairy has advanced cardiac monitoring system...,"['appairy', 'has', 'advanced', 'cardiac', 'mon...","['appairy', 'advanced', 'cardiac', 'monitoring...",appairy advanced cardiac monitoring systems co...
7,8,ARIZ Precision Medicine,"Davis, CA",ariz precision medicine ariz developing potent...,ARIZ Precision Medicine ARIZ is developing pot...,"['ariz', 'precision', 'medicine', 'ariz', 'is'...","['ariz', 'precision', 'medicine', 'ariz', 'dev...",ariz precision medicine ariz developing potent...
8,9,Articulate Labs,"Dallas, TX",articulate labs developed kneestimtm medical d...,Articulate Labs has developed KneeStimTM a med...,"['articulate', 'labs', 'has', 'developed', 'kn...","['articulate', 'labs', 'developed', 'kneestimt...",articulate labs developed kneestimtm medical d...
9,10,Avitus Orthopaedics,"Shelton, CT",avitus orthopaedics inc medical device startup...,Avitus Orthopaedics Inc is a medical device st...,"['avitus', 'orthopaedics', 'inc', 'is', 'a', '...","['avitus', 'orthopaedics', 'inc', 'medical', '...",avitus orthopaedics inc medical device startup...


In [91]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
                strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                ngram_range=(1, 3),
                stop_words = 'english')
    
    # Filling NaNs with empty string
df1['Overview'] = df1['Overview'].fillna('')
     
    # Fitting the TF-IDF on the 'Overview' text
tfv_matrix = tfv.fit_transform(df1['Overview'])
     
    # Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)
    
 
    # Reverse mapping of indices and movie titles
indices = pd.Series(cleaned_df.index, index=cleaned_df['Name']).drop_duplicates()
    
    
company = 'CareSuites'
index_no = indices[company]
    
    
list(enumerate(sig[indices[company]]))
    
sorted(list(enumerate(sig[indices[company]])), key=lambda x: x[1], reverse=True)

[(15, 0.7649602983630058),
 (19, 0.7625728315940541),
 (17, 0.7623769026351848),
 (3, 0.7622474517206358),
 (31, 0.7622273311648894),
 (30, 0.7621493162246756),
 (6, 0.7621389220398157),
 (20, 0.7620996146939945),
 (18, 0.7620681439987322),
 (36, 0.7620589777381779),
 (10, 0.7620500547470184),
 (1, 0.7619713364883663),
 (38, 0.7619672288591862),
 (32, 0.7619061716398632),
 (2, 0.7619033228949184),
 (11, 0.7618791771969526),
 (33, 0.7618422182834024),
 (37, 0.7617911563600828),
 (16, 0.7617788191679945),
 (12, 0.7617763351696042),
 (4, 0.7617752178614239),
 (29, 0.7617457735382034),
 (5, 0.7617433978210391),
 (24, 0.7617151312445486),
 (8, 0.7617086665443196),
 (26, 0.7617027171914853),
 (25, 0.7616899811701587),
 (34, 0.7616883998564108),
 (28, 0.7616879770418454),
 (13, 0.7616873920664495),
 (0, 0.7616697465416131),
 (7, 0.7615941559557649),
 (9, 0.7615941559557649),
 (14, 0.7615941559557649),
 (21, 0.7615941559557649),
 (22, 0.7615941559557649),
 (23, 0.7615941559557649),
 (27, 0.761

In [93]:
xl = pd.ExcelFile('data_for_content_based - Ext.xlsx')
df = xl.parse('Sheet1')
    
rslt_df = df[df['Name'] == "AMChart"].Overview.to_string()

rslt_df

'3    The AmChart platform was created to bring thes...'